# Naive Bayes Classifier

## Multinomial Naive Bayes

In [1]:
class Multinomial_NB:
    def __init__(self, alpha = 1.0):
        '''Initialization function to instantiate the class.'''
        self.alpha = alpha
    def fit(self, X, Y):
        '''
        Input
        X: ndarray, A numpy array with rows representing data samples and columns
        representing numerical features.
        Y: ndarray, A 1D numpy array with labels corresponding to each row of the feature
        matrix X.
        '''
        # define basic parameters
        self.X = X
        self.y = Y
        self.classes = np.unique(Y,return_counts = True)
        self.classes_num = len(self.classes[0]) # the number of classes
        N, J = X.shape # J - feature, the number of words, N - the number of sample size
        #alpha: parameteres of the distirbution
        sum_alpha_prior = self.classes_num*alpha
        # calculate the prior probabilty
        for i in range(self.classes_num):
            counts_classes = self.classes[1][i]
            self.pi[i] = (self.alpha+counts_classes)/(N+sum_alpha_prior)
        # calculate the conditional probabilty
        sum_alpha_likelihood = J*alpha
        for i in range(self.classes_num):
            X_i = X[np.nonzeros(Y==self.classes[0][i])[0],:]
            M_J_i = X_i.sum(axis = 0)
            M_i = M_J_i.sum()
            for j in range(J):
                self.theta[i][j] = (self.alpha+M_J_i)/(sum_alpha_likelihood+M_i)
    def predit(self,X):
        '''
        This method performs classification on an array of test vectors X. 
        Returns:
        1D array (column vector) of predictions for each row in X.
        '''
        pred = self.predict_log_proba(X)
        y_pred = np.zeros(X.shape)
        for i in range(X.shape[0]):
            # prediction by row (sample)
            y_pred[i] = self.classes[0][np.argmax(pred[i])]
        return y_pred
    def predict_log_proba(self,X):
        '''
        This method returns log-probability estimates for the test matrix X.
        Arguments:
        X : ndarray
        A numpy array containing samples to be used for prediction. Its rows
        represent data samples and columns represent numerical features.
        Returns:
        A numpy array that contains log-probability of the samples (unnormalized
        log posteriors) for each class in the model. The number rows are equal to
        the rows in X and number of columns are equal to the number of classes.
        '''
        N, J = X.shape # J - feature, the number of words, N - the number of sample size
        classes_num = len(self.classes[0]) #the number of classess
        prob = np.zeros(shape=(N, classes_num)) #probabilty matrix
        for k in range(classes_num): #in this case, its 2
            for i in range(N):
                summation = 0
                for j in range(J):
                    summation += np.log(self.theta[k,j]**X[i,j])
                prob[i,k] = np.log(self.pi[k])+summation
        return prob
    def predict_proba(self,X):
        N, J = X.shape # J - feature, the number of words, N - the number of sample size
        classes_num = len(self.classes[0]) #the number of classess
        prob = np.zeros(shape=(N, classes_num)) #probabilty matrix
        
        for k in range(classes_num):
            for i in range(N):
                p = 1
                for j in range(J):
                    p *= self.theta[k,j]**X[i,j]
                prob[i,k] = self.pi[k]*p
        return prob